<a href="https://colab.research.google.com/github/stevehooker/welsh-dragons/blob/main/notebooks/comfyui_colab_optimised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ComfyUI on Google Colab (Optimised)

**⚡ Fast startup (~5 minutes after first run)**

**Features:**
- Google Drive integration for persistent storage
- ComfyUI Manager already installed
- Cloudflare tunnel for external access
- Colab Secrets integration for API tokens
- Optimised dependency installation (only installs what's missing)

**Setup Instructions:**
1. Enable GPU: Runtime → Change runtime type → T4 GPU
2. Add secrets (optional): Tools → Secrets → Add your API keys
3. Run all cells: Runtime → Run all

## 0. Configure Secrets (Optional)

Store your API keys securely using Colab Secrets:
1. Click the 🔑 key icon in the left sidebar
2. Add your secrets:
   - `ANTHROPIC_API_KEY` - For Claude integration
   - `OPENAI_API_KEY` - For OpenAI models
   - `HF_TOKEN` - For Hugging Face private models
3. Toggle "Notebook access" for each secret

Secrets are stored in your Google account, not in the notebook file.

In [ ]:
#@title Load Secrets (if available)
import os

try:
    from google.colab import userdata
    
    secrets_found = []
    
    for key in ['ANTHROPIC_API_KEY', 'OPENAI_API_KEY', 'HF_TOKEN']:
        try:
            value = userdata.get(key)
            if value:
                os.environ[key] = value
                secrets_found.append(key)
        except:
            pass
    
    if secrets_found:
        print(f"✓ Loaded secrets: {', '.join(secrets_found)}")
    else:
        print("ℹ️  No secrets found. Add them in the 🔑 Secrets panel if needed.")
        
except Exception as e:
    print(f"ℹ️  Secrets not available: {e}")

## 1. Mount Google Drive

Mount your Google Drive where ComfyUI and models are already stored.

In [ ]:
#@title Mount Google Drive

print("Mounting Google Drive...")
from google.colab import drive
drive.mount('/content/drive')

WORKSPACE = "/content/drive/MyDrive/ComfyUI"
%cd /content/drive/MyDrive

print(f"✓ Workspace: {WORKSPACE}")

## 2. ComfyUI Setup Check

**ComfyUI is already installed** - this just verifies it exists.

Only enable update if you need the latest ComfyUI version.

In [ ]:
#@title Check ComfyUI Installation
#@markdown Update ComfyUI to latest version? (Usually not needed)
UPDATE_COMFY_UI = False  #@param {type:"boolean"}

import os

if not os.path.exists(WORKSPACE):
    print("❌ ComfyUI not found on Drive!")
    print("Please install ComfyUI first using the standard setup notebook.")
else:
    print("✓ ComfyUI found")
    %cd {WORKSPACE}
    
    if UPDATE_COMFY_UI:
        print("  Updating ComfyUI...")
        !git pull origin master 2>/dev/null || git pull origin main
        print("  ✓ Updated!")

## 3. Install Dependencies (Smart)

**⚡ OPTIMISED:** Only installs packages missing from Colab's base image.

Skips: torch, torchvision, torchaudio, transformers, Pillow, scipy, tqdm (already in Colab)

Installs: einops, safetensors, aiohttp, kornia, torchsde and other ComfyUI-specific packages.

In [ ]:
#@title Install Missing Dependencies Only

print("=== Installing Missing Dependencies ===")
print("Skipping packages already in Colab...\n")

%cd {WORKSPACE}

# These are NOT in Colab's base image (using --no-deps to avoid reinstalling torch)
!pip install -q --no-deps einops safetensors aiohttp pyyaml psutil tokenizers kornia spandrel soundfile av sentencepiece

# torchsde needs its dependencies (especially trampoline)
!pip install -q torchsde

# Upgrade if too old
!pip install -q --upgrade transformers accelerate

# ComfyUI requirements (skip duplicates)
!pip install -q -r requirements.txt --no-deps 2>/dev/null || true

# Frontend
!pip install -q --upgrade comfyui-frontend-package

print("\n✓ Dependencies installed!")

## 3.5. Verify GPU

Quick check that PyTorch can see the GPU.

In [ ]:
import torch

print(f"PyTorch {torch.__version__}")

if torch.cuda.is_available():
    print(f"CUDA {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print("\n✓ GPU ready!")
else:
    print("\n❌ No GPU! Go to: Runtime > Change runtime type > T4 GPU")

## 4. ComfyUI Manager Check

**Manager is already installed** - this just verifies it exists.

Usually you don't need to update it.

In [ ]:
#@title Check ComfyUI Manager
#@markdown Update Manager to latest version? (Usually not needed)
UPDATE_MANAGER = False  #@param {type:"boolean"}

import os
manager_path = f"{WORKSPACE}/custom_nodes/ComfyUI-Manager"

if os.path.exists(manager_path):
    print("✓ ComfyUI Manager found")
    
    if UPDATE_MANAGER:
        print("  Updating Manager...")
        %cd {manager_path}
        !git pull
        print("  ✓ Updated!")
else:
    print("⚠️  Manager NOT found")
    print("Install it from ComfyUI Manager menu in the web UI")

## 4.5. Install Custom Node Dependencies

**⚡ FAST:** Essential packages only (5-10 seconds)

These Python packages are temporary - they disappear when the Colab VM restarts,
so we reinstall them each session. Your custom nodes on Drive are permanent.

In [ ]:
#@title Essential Custom Node Packages
#@markdown Quick install (5-10 seconds)
INSTALL_ESSENTIAL = True  #@param {type:"boolean"}

#@markdown ---
#@markdown **Advanced:** Full restore of ALL dependencies? (30-40 minutes!)
FULL_RESTORE = False  #@param {type:"boolean"}

if INSTALL_ESSENTIAL:
    print("=== Installing Essential Packages ===")
    
    !pip install -q ffmpeg-python deepdiff comfyui-types gguf piexif \
        transparent-background simpleeval iopath ollama anthropic segment-anything \
        GitPython
    
    print("✓ Essential packages installed!")

if FULL_RESTORE:
    print("\n=== Full Restore (30-40 minutes) ===")
    %cd {WORKSPACE}
    !python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies
    print("✓ Full restore complete!")

## 5. Start ComfyUI

**⚡ FAST:** Cloudflared URL appears in ~10 seconds.

Click the URL below to access ComfyUI!

In [ ]:
import subprocess
import threading
import time
import socket
import re

# Download cloudflared (cached between runs)
![ ! -f ~/cloudflared-linux-amd64.deb ] && wget -q -O ~/cloudflared-linux-amd64.deb https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb 2>/dev/null

def start_tunnel(port=8188):
    """Start cloudflared and print URL quickly"""
    print("Waiting for ComfyUI server...")
    
    for _ in range(60):
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(1)
            result = sock.connect_ex(('127.0.0.1', port))
            sock.close()
            if result == 0:
                break
        except:
            pass
        time.sleep(0.5)
    
    print("\n" + "=" * 70)
    print("Starting cloudflared tunnel...")
    
    proc = subprocess.Popen(
        ["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
        bufsize=1
    )
    
    # Extract URL immediately
    url_found = False
    for line in proc.stderr:
        if "trycloudflare.com" in line and not url_found:
            match = re.search(r'https://[a-z0-9-]+\.trycloudflare\.com', line)
            if match:
                url = match.group(0)
                print(f"\n🌐 ACCESS COMFYUI HERE:\n{url}\n")
                print("=" * 70 + "\n")
                url_found = True
                break

# Start tunnel in background
tunnel_thread = threading.Thread(target=start_tunnel, daemon=True)
tunnel_thread.start()

# Start ComfyUI
print("\n" + "=" * 70)
print("Starting ComfyUI...")
print("=" * 70 + "\n")

%cd {WORKSPACE}
!python main.py --dont-print-server

## Tips & Troubleshooting

**Speed:**
- First run: ~8-10 min (downloading dependencies)
- Later runs: ~3-5 min (everything cached)
- Leave UPDATE options disabled unless you need them

**API Keys:**
- Store in Colab Secrets (🔑 icon)
- Enable "Notebook access"
- Auto-loaded in Section 0

**Issues:**
- "No GPU": Enable T4 GPU in Runtime settings
- Custom node errors: Try FULL_RESTORE in Section 4.5
- Slow startup: Don't enable UPDATE options unless needed

**Storage:**
- **Permanent** (Drive): ComfyUI files, models, custom nodes, workflows
- **Temporary** (Colab VM): Python packages (reinstalled each session)